In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/doc2dial/doc2dial_testset/DEFAULT/eval.csv')

In [10]:
def example(index):
    # print(df.loc[index])
    print('Question: ')
    print(df.loc[index]['question'])
    print('True Answer:')
    print(df.loc[index]['answer'])
    print('Model Answer:')
    print(df.loc[index]['model_answer'])
    print('True Ref:')
    print(df.loc[index]['true_ref_str'])
    print('retrived_doc:')
    print(df.loc[index]['retrived_doc'])

    
def analys(df):
    answer_f1_avg = df['answer_f1'].mean()
    answer_prec = df['answer_prec'].mean()
    answer_recall = df['answer_recall'].mean()
    # autoais_retrevied = df['autoais_retrevied(model_answer)'].mean()
    autoais_retrevied = len(df[df['autoais_retrevied(model_answer)'] == '1'])/len(df)
    att_f1 = df['att_f1'].mean()
    att_prec = df['att_prec'].mean()
    att_recall = df['att_recall'].mean()
    autoais_true_answer = len(df[df['autoais_true_answer'] == '1'])/len(df)
    print(f'answer_f1_avg: {answer_f1_avg}')
    print(f'answer_prec: {answer_prec}')
    print(f'answer_recall: {answer_recall}')
    print(f'autoais_retrevied: {autoais_retrevied}')
    print(f'att_f1: {att_f1}')
    print(f'att_prec: {att_prec}')
    print(f'att_recall: {att_recall}')
    print(f'autoais_true_answer: {autoais_true_answer}')
    

In [7]:
example(2)

Question: 
What if my insurance lapsed too?
True Answer:
You'll need to either bring a copy of your current, new, insurance or pay the un-insured fee.
Please make sure to bring all documentation needed to a local branch to expedite this process.
Model Answer:
I would have to pay for the plates
True Ref:
Because we all pay indirectly for crashes involving uninsured motorists ,  New York State requires every motorist to maintain auto insurance every single day a vehicle is registered.  DMV works with insurance companies to electronically monitor your insurance coverage ,  When that happens , 
retrived_doc:
possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.


In [5]:
analys(df)

answer_f1_avg: 0.12677739186135162
answer_prec: 0.27770991880233487
answer_recall: 0.10309733293597707
autoais_retrevied: 0.2333000997008973
att_f1: 0.2258884523339757
att_prec: 0.21296288706875702
att_recall: 0.4704671315478364
autoais_true_answer: 0.21073558648111332


In [12]:
df = pd.read_csv('data/doc2dial/doc2dial_testset/doc2dial_500_top5/eval.csv')

In [13]:
analys(df)

answer_f1_avg: 0.1416277043466649
answer_prec: 0.28963087890552514
answer_recall: 0.12242884258533641
autoais_retrevied: 0.33399602385685884
att_f1: 0.12060043167434409
att_prec: 0.0682965806981322
att_recall: 0.7944171008904347
autoais_true_answer: 0.352882703777336
